In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
from dash_extensions import BeforeAfter

# Load data
planet_info = pd.read_csv('modified_planets.csv')
coordinates = pd.read_csv('planetary_coordinates_2025_01_16.csv')

# Functions
def calculate_angle(revolution_days, velocity, perimeter, time_difference):
    distance_travelled = velocity * time_difference
    angle_moved = (distance_travelled / perimeter) * 360
    return angle_moved

def get_new_coordinates(user_date_str):
    user_date = datetime.strptime(user_date_str, "%Y-%m-%d")
    date_16jan2025 = datetime(2025, 1, 16)

    time_difference = (user_date - date_16jan2025).days
    new_coordinates = []

    for _, planet_row in planet_info.iterrows():
        planet = planet_row['Planet']
        revolution_days = planet_row['Orbital Period (days)']
        velocity = planet_row['Orbital Velocity (km/s)'] * 86400
        perimeter = planet_row['Perimeter (10^6)(km)'] * 1e6
        inclination = np.radians(planet_row['Orbital Inclination (degrees)'])

        initial_coords = coordinates[coordinates['Planet'] == planet].iloc[0]
        x, y, z = initial_coords['X (AU)'], initial_coords['Y (AU)'], initial_coords['Z (AU)']

        angle_moved = calculate_angle(revolution_days, velocity, perimeter, time_difference)
        angle_radians = np.radians(angle_moved)

        new_x = x * np.cos(angle_radians) - y * np.sin(angle_radians)
        new_y = x * np.sin(angle_radians) + y * np.cos(angle_radians)
        new_z = z * np.cos(inclination)

        new_coordinates.append({
            'Planet': planet,
            'New_X (AU)': new_x,
            'New_Y (AU)': new_y,
            'New_Z (AU)': new_z
        })

    new_coords_df = pd.DataFrame(new_coordinates)
    return new_coords_df

# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Enhanced Solar System Viewer with Tailwind CSS"

# Include Tailwind CSS using Dash Extensions
app.index_string = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
    <title>Solar System Viewer</title>
</head>
<body class="bg-gray-900 text-white">
    <div id="app">
        {%app_entry%}
    </div>
    <footer>
        {%config%}
        {%scripts%}
        {%renderer%}
    </footer>
</body>
</html>
'''


# Layout with enhanced Tailwind CSS styles
app.layout = html.Div([
    html.H1("Solar System Orbit Viewer", className="text-4xl text-center font-bold mb-6 text-yellow-500"),
    
    html.Div([
        html.Label("Enter the date (YYYY-MM-DD):", className="block text-lg font-medium mb-2"),
        dcc.Input(id='user_date', type='text', placeholder="YYYY-MM-DD", className="border rounded p-2 mb-4 w-1/2"),
        html.Button("Calculate", id='calculate_button', n_clicks=0, className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"),
    ], className="text-center mb-6"),

    html.Div(id='status_output', className="text-center text-red-500 mb-6"),

    html.Div([
        dcc.Graph(id='solar_system_plot', style={'height': '80vh', 'width': '100%'})
    ], className="flex justify-center")
])

# Callback for updating the solar system plot
@app.callback(
    [Output('status_output', 'children'),
     Output('solar_system_plot', 'figure')],
    [Input('calculate_button', 'n_clicks')],
    [State('user_date', 'value')]
)
def update_solar_system(n_clicks, user_date_str):
    if n_clicks > 0:
        if not user_date_str:
            return "Please enter a valid date in YYYY-MM-DD format.", go.Figure()

        try:
            new_coords = get_new_coordinates(user_date_str)
            
            # Create a solar system plot
            fig = go.Figure()

            # Add the Sun at the center
            fig.add_trace(go.Scatter(
                x=[0], y=[0], mode='markers+text',
                marker=dict(size=40, color='yellow'),
                name='Sun',
                text='Sun',
                textposition='bottom center'
            ))

            # Add each planet's current position with improved markers and spacing
            for _, row in new_coords.iterrows():
                fig.add_trace(go.Scatter(
                    x=[row['New_X (AU)']],
                    y=[row['New_Y (AU)']],
                    mode='markers+text',
                    marker=dict(size=15, symbol='circle'),
                    name=row['Planet'],
                    text=row['Planet'],
                    textposition='top center'
                ))

            # Add orbit lines for each planet with proper scaling and spacing
            for _, planet_row in coordinates.iterrows():
                orbit_x = [planet_row['X (AU)'] * np.cos(t) - planet_row['Y (AU)'] * np.sin(t) for t in np.linspace(0, 2*np.pi, 100)]
                orbit_y = [planet_row['X (AU)'] * np.sin(t) + planet_row['Y (AU)'] * np.cos(t) for t in np.linspace(0, 2*np.pi, 100)]
                fig.add_trace(go.Scatter(
                    x=orbit_x, y=orbit_y,
                    mode='lines',
                    line=dict(dash='dot', width=1, color='gray'),
                    name=f"{planet_row['Planet']} Orbit",
                    showlegend=False
                ))

            # Update layout for better visualization and alignment
            fig.update_layout(
                title="Solar System Visualization",
                xaxis=dict(title="X (AU)", range=[-50, 50], zeroline=False),
                yaxis=dict(title="Y (AU)", range=[-50, 50], zeroline=False),
                paper_bgcolor="#1a202c",
                plot_bgcolor="#2d3748",
                font=dict(color="white"),
                showlegend=True
            )

            return "Coordinates successfully updated!", fig
        except Exception as e:
            return f"An error occurred: {str(e)}", go.Figure()
    return "Enter a date and click 'Calculate'.", go.Figure()

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
